In [1]:
import plotly.express as px
from utils.ops import load_yaml
from  pathlib import Path
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Preparing

In [2]:
site = 3
site_label  = 'AM'

In [3]:
cfg = load_yaml(f'cfg{site}.yaml')
site_cfg = load_yaml(f'site_{site}.yaml')

preparation_params = cfg['preparation_params']
paths_params = cfg['paths']
general_params = cfg['general_params']
experiments_paths = general_params['experiments_folders']

images_path = Path('images')
images_path.mkdir(exist_ok=True)

## Functions

In [8]:
def results2(experiments):
    exp_results = []
    for exp_i in experiments:
        #experiment_params = cfg['experiments'][f'exp_{experiments[exp_i]}']
        exp_path = Path(paths_params['experiments']) / f'exp_{exp_i}'
        experiment_params = cfg['experiments'][f'exp_{exp_i}']

        results_path = exp_path / experiments_paths['results']
        final_results_file = results_path / f'results_{exp_i}.data'

        result_df = pd.read_pickle(final_results_file)

        exp_results.append(
            {
                'name': f'{experiment_params["short"]}', # ({exp_i})',
                'cloud_cond': 'All Pixels',
                'f1': result_df['global_f1'][0],
                'precision': result_df['global_precision'][0],
                'recall': result_df['global_recall'][0],
                'group_model': experiment_params['group_model'],
                'prev_def': 'False' if '[No-prev-def]' in experiment_params['short'] else 'True'
            }
        )

        exp_results.append(
            {
                'name': f'{experiment_params["short"]}', # ({exp_i})',
                'cloud_cond': 'No Cloud',
                'f1': result_df['no_cloud_f1'][0],
                'precision': result_df['no_cloud_precision'][0],
                'recall': result_df['no_cloud_recall'][0],
                'group_model': experiment_params['group_model'],
                'prev_def': 'False' if '[No-prev-def]' in experiment_params['short'] else 'True'
            }
        )

        exp_results.append(
            {
                'name': f'{experiment_params["short"]}', # ({exp_i})',
                'cloud_cond': 'Cloudy',
                'f1': result_df['cloud_f1'][0],
                'precision': result_df['cloud_precision'][0],
                'recall': result_df['cloud_recall'][0],
                'group_model': experiment_params['group_model'],
                'prev_def': 'False' if '[No-prev-def]' in experiment_params['short'] else 'True'
            }
        )


        
    return pd.DataFrame(exp_results)

## Data preparing

In [9]:
experiments = [
    6,
    12,
    2,
    13,
    56,
    62,
    52,
    63
    
]
exp_results = results2(experiments)

In [10]:
exp_results

,name,cloud_cond,f1,precision,recall,group_model,prev_def
0,CNN-OPT [Free-cloud],All Pixels,0.482011,0.319568,0.980338,CNN,True
1,CNN-OPT [Free-cloud],No Cloud,0.482013,0.319569,0.980338,CNN,True
2,CNN-OPT [Free-cloud],Cloudy,0.347826,0.210526,1.000000,CNN,True
3,CNN-OPT [Free-cloud][No-prev-def],All Pixels,0.842016,0.927886,0.770693,CNN,False
4,CNN-OPT [Free-cloud][No-prev-def],No Cloud,0.842019,0.927886,0.770698,CNN,False
5,CNN-OPT [Free-cloud][No-prev-def],Cloudy,NaN,NaN,0.000000,CNN,False
6,CNN-SAR [12-AverageSAR],All Pixels,0.670167,0.633225,0.711687,CNN,True
7,CNN-SAR [12-AverageSAR],No Cloud,0.674401,0.640529,0.712055,CNN,True
8,CNN-SAR [12-AverageSAR],Cloudy,0.650544,0.600328,0.709927,CNN,True
9,CNN-SAR [12-AverageSAR][No-prev-def],All Pixels,0.219734,0.793343,0.127528,CNN,False


In [14]:
exp_results.loc[(exp_results['name'].str.contains('-SAR') & exp_results['cloud_cond'].str.contains('Cloudy')), 'f1'] = np.nan
exp_results.loc[(exp_results['name'].str.contains('-SAR') & exp_results['cloud_cond'].str.contains('No Cloud')), 'f1'] = np.nan

#cnn_exp_results = exp_results[exp_results['group_model'] == 'CNN']
cnn_exp_results = exp_results
cnn_max = cnn_exp_results[cnn_exp_results['name'].str.contains('\[Free-cloud\]')]['f1'].max()
cnn_exp_results.loc[(cnn_exp_results['name'].str.contains('\[Free-cloud\]') & exp_results['cloud_cond'].str.contains('No Cloud')), 'f1'] = np.nan
cnn_exp_results.loc[(cnn_exp_results['name'].str.contains('\[Free-cloud\]') & exp_results['cloud_cond'].str.contains('Cloudy')), 'f1'] = np.nan

x_lim1 = 2/4.

cnn_exp_results = cnn_exp_results.replace(' \[Free-cloud\]', '', regex=True)
cnn_exp_results = cnn_exp_results.replace('\[No-prev-def\]', '', regex=True)
cnn_exp_results = cnn_exp_results.replace('\[12-AverageSAR\]', '', regex=True)

fig = px.bar(cnn_exp_results, x="name", y="f1",
             color='prev_def', barmode='group',
             labels = {
                 'f1': 'F1-Score',
                 'name': 'Model',
                 'cloud_cond': 'Cloud Condition',
                 'prev_def': 'Previous Deforestation Map'
                 },
             height=400,
             width= 800)

fig.add_hrect(y0=0, y1=1, x0 = 0.0, x1 = x_lim1, line_width=0, fillcolor="green", opacity=0.15, layer = 'below')
fig.add_hrect(y0=0, y1=1, x0 = x_lim1, x1 = 1, line_width=0, fillcolor="purple", opacity=0.15, layer = 'below')
fig.add_annotation(text='CNN Models', x=0.5, y=1.05, showarrow = False)
fig.add_annotation(text='Transformer-based Models', x=2.5, y=1.05, showarrow = False)

fig.update_layout(title_text=f'Previoius Deforestation Map Improvement (Site {site})', title_x=0.5)
fig.show()
fig.write_image(f'images/site_{site}_def_prev.png')